### Rerank

WPS process for the [rerank.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/rerank.R) from [ClimDown](https://github.com/pacificclimate/ClimDown)

**Quantile Reranking** fixes bias introduced by the [Climate Analogues (CA)](https://github.com/pacificclimate/ClimDown/blob/master/R/CA.R) step by re-applying a simple quantile mapping bias correction at each grid box

In [1]:
from birdy import WPSClient
from netCDF4 import Dataset
from pkg_resources import resource_filename
from wps_tools.testing import get_target_url
from wps_tools.utils import copy_http_content
from tempfile import NamedTemporaryFile
import requests
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [3]:
chickadee = WPSClient(url)

### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.rerank?

Signature:
chickadee.rerank(
    obs_file,
    varname=None,
    qdm_file=None,
    num_cores='4',
    loglevel='INFO',
    out_file=None,
    analogues_file=None,
)
Docstring:
Quantile Reranking fixes bias introduced by the Climate Analogues step

Parameters
----------
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
varname : string
    Name of the NetCDF variable to downscale (e.g. 'tasmax')
out_file : string
    Filename to create with the climate imprint outputs
num_cores : {'1', '2', '3', '4'}positiveInteger
    The number of cores to use for parallel execution
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
qdm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of output from QDM step
analogues_file : string
    R object containing the analogues produced from the CA step (suffix .rds)

R

### Run the rerank process

In [5]:
out_file = NamedTemporaryFile(suffix=".nc", prefix="output_", dir="/tmp", delete=True)

output = chickadee.rerank(
    qdm_file=resource_filename("tests", "/data/QDM_expected_output.nc"), 
    obs_file=resource_filename("tests", "/data/tiny_obs.nc"), 
    varname="tasmax", 
    out_file=out_file.name,
    num_cores=2,
    analogues_object=resource_filename("tests", "/data/analogues.rds"),
)

out_file.close()

output_data = output.get()[0]

### Test for expected output

In [6]:
with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    output_data = Dataset(copy_http_content(output_data, tmp_file))
    expected_data = Dataset(resource_filename("tests","/data/bccaq_expected_output.nc"))
    for key, value in expected_data.dimensions.items():
        assert str(output_data.dimensions[key]) == str(value)